In [ ]:
%%sh
# export train service to gcs
gsutil cp -r /home/jovyan/work/<path-to-service>/train-service.py gs://jupyter-storage/<path-in-gcs>/train-service.py

In [ ]:
from kubernetes import client, config

### CREATE TRAINING POD

In [ ]:
config.load_incluster_config()

v1 = client.CoreV1Api()

pod = client.V1Pod()
spec = client.V1PodSpec()
pod.metadata = client.V1ObjectMeta(name="train")

container = client.V1Container()
container.image = "registry.inovex.de:4567/inovex-kubeyard/docker-jupyter:latest"
container.args = ["/bin/bash", "-c", "gsutil cp gs://jupyter-storage/<path-in-gcs>/train-service.py . && ipython -- train-service.py"]
container.name = "train"

spec.containers = [container]
pod.spec = spec

v1.create_namespaced_pod(namespace="kubeyard", body=pod)

## READ POD LOG

In [ ]:
config.load_incluster_config()

v1 = client.CoreV1Api()
response = v1.read_namespaced_pod_log(name="train", namespace="kubeyard")

print(response)

## DELETE TEST POD

In [ ]:
config.load_incluster_config()

v1 = client.CoreV1Api()

v1.delete_namespaced_pod(name="train", namespace="kubeyard", body=client.V1DeleteOptions())